This notebook will use evaluation code to make some plots of MARS's performance on test data.

In [1]:
%matplotlib inline

from evaluation import evaluation
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



First, we need to run our trained model on the test set. For this you'll need:
* the path to the test tfrecord
* the path to the trained model checkpoint
* the path to `config_test.yaml`
* the path to a directory where you'd like your summary figures/files to be saved

In [ ]:
# tfrecords       = ['P:\\cristina_code_original\\tfrecords\\test_dataset-00000-of-00001']
# checkpoint_path = 'P:\\cristina_code_original\\results_keypoints\\top_fixed'
# summary_dir     = 'P:\\MARS_eval_test\\'
# config          = 'P:\\cristina_code_original\\config_files\\config_test.yaml'

tfrecords       = ['D:\\MARS_data\\tfrecords\\tfrecords_pose_top\\test_dataset-00000-of-00001']
checkpoint_path = 'X:\\cristina_code_original\\results_keypoints\\top_fixed'
summary_dir     = 'D:\\MARS_eval_test\\top\\'
config          = 'D:\\MARS_data\\tfrecords\\tfrecords_pose_top\\config_test.yaml'

# tfrecords       = ['D:\\MARS_data\\tfrecords\\tfrecords_pose_front\\test_dataset-00000-of-00001']
# checkpoint_path = 'X:\\cristina_code_original\\results_keypoints\\front'
# summary_dir     = 'D:\\MARS_eval_test\\front\\'
# config          = 'D:\\MARS_data\\tfrecords\\tfrecords_pose_front\\config_test.yaml'

# optional inputs
view                = 'top'    # top or front camera
max_iterations      = 0        # only run on the first X images (set to 0 to run on all data)- useful for debugging
prep_cocoEval       = True     # whether to save evaluation results for future analysis with MARSeval

evaluation.process_tfrecord(tfrecords, checkpoint_path, summary_dir, config,
                            view=view,
                            max_iterations=max_iterations,
                            prep_cocoEval=prep_cocoEval)

X:\cristina_code_original\results_keypoints\top_fixed
is now:X:\cristina_code_original\results_keypoints\top_fixed\model.ckpt-539952
INFO:tensorflow:Restoring parameters from X:\cristina_code_original\results_keypoints\top_fixed\model.ckpt-539952
Found model for global step: 539952
Step: 0, Time/image network (ms): 608.7
Step: 1, Time/image network (ms): 378.2
Step: 2, Time/image network (ms): 374.7
Step: 3, Time/image network (ms): 381.2
Step: 4, Time/image network (ms): 373.3
Step: 5, Time/image network (ms): 356.7


Okay now let's look at our performance:

In [ ]:
fid = summary_dir + 'MARS_performance_pose.json'

coco = evaluation.eval_coco(fid)
coco=coco[0] # all-keypoints evaluation
oks=[]
for i in coco.params.imgIds:
    oks.append(coco.computeOks(i,1)[0][0])
    
coco = evaluation.eval_coco(fid,fixedSigma='narrow')
oks2=[]
for i in coco.params.imgIds:
    oks2.append(coco.computeOks(i,1)[0][0])

# sc=0.05
# sns.kdeplot(oks, bw=sc)
# sns.kdeplot(oks2, bw=sc)

# sns.ecdfplot(oks)
# sns.ecdfplot(oks2)

counts,bins=np.histogram(oks,20,(0,1))
plt.bar(bins[1:],counts,0.9/20)

counts,bins=np.histogram(oks2,20,(0,1))
plt.bar(bins[1:],counts,0.9/20)

plt.savefig(summary_dir + 'OKS_compare_bar.pdf')